In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_baza = pd.read_csv("utitles_cats.tsv", sep="\t", dtype=str)

In [ ]:
# посмотрим несколько строк датафрейма и его размерность

df_baza

In [ ]:
# посмотрим сколько пустых значений содержат колонки нашего датафрейма

df_baza.isnull().sum()

In [ ]:
# удалим строки, где в колонке "is_drug" стоит значение "False"
mask = df_baza['is_drug'] == 'False'
df_baza.drop(df_baza[mask].index, inplace=True)

# там, где не указан manuf_cat, перенесем данные из колонки manuf
df_baza['manuf_cat'].fillna(df_baza['manuf'], inplace=True)

# удаляем строки, где содержатся пустые значения в колонках title, manuf, title_cat, manuf_cat
df_baza = df_baza.loc[df_baza['manuf_cat'] != 0]
df_baza.dropna(subset=['title', 'title_cat', 'manuf_cat'], inplace=True)

In [ ]:
df_baza.head(10)

In [ ]:
# организуем процесс выборки из массива данных 1500 уникальных категорий и по каждой из категорий возьмем от 4 до 20 значений title,
# при этом первое значение title - будет использоваться в списке аптеки,
# второй значение title для той же категории - будет использоваться в списке поставщика
# остальные значения title для той же категории - будут использоваться в списке study для проведения обучения.

from tqdm import tqdm

duplicates = df_baza[df_baza.duplicated('title', keep=False)]
unique_titles = duplicates.drop_duplicates('title')['title'].tolist()

indexes_to_drop = df_baza[df_baza['title'].isin(unique_titles)].index

df_baza_unique = df_baza.drop(indexes_to_drop)

categories_with_enough_titles = df_baza_unique.groupby('title_cat').filter(lambda x: len(x) >= 4)

random_categories = categories_with_enough_titles['title_cat'].drop_duplicates().sample(n=1500, random_state=1)

df = pd.DataFrame() 

for cat in tqdm(random_categories, desc='Фильтрация категорий'):
    sampled_df = df_baza_unique[df_baza_unique['title_cat'] == cat].sample(n=min(20, len(df_baza_unique[df_baza_unique['title_cat'] == cat])), random_state=1)
    df = pd.concat([df, sampled_df], ignore_index=True)

# Проверяем, достигнуто ли требуемое количество уникальных категорий
if len(df['title_cat'].unique()) == 1500:
    print('Требуемое количество уникальных категорий достигнуто')
else:
    print('Требуемое количество уникальных категорий не достигнуто: ', len(df['title_cat'].unique()))

In [ ]:
# проводим предобработку данных (в части знаков препинания и лишних пробелов)

import re
import string
from tqdm.auto import tqdm
tqdm.pandas()

def replace_punctuation(title):
    title = re.sub(r'[^\w\s]', '.', title)
    title = re.sub(r'\.{2,}', '.', title)
    return title

def remove_multiple_spaces(title):
    return re.sub(r'\s+', ' ', title, flags=re.I)

df['title'] = df['title'].progress_apply(lambda title: remove_multiple_spaces(replace_punctuation(title)).lower())

df.head(20)

In [ ]:
# дополнительно проведем предобработку данных по стоп-словам с добавлением в список слов "акция", "скидка", "подарок", 'внимание', 'неопознанный'

from nltk import word_tokenize
from nltk.corpus import stopwords

# Загрузка стоп-слов для русского языка, добавление в стоп-слова слов "акция", "скидка", "подарок", 'внимание', 'неопознанный'
russian_stopwords = stopwords.words('russian')
russian_stopwords.extend(['скидка', 'подарок', 'акция','внимание', 'неопознанный'])

def filter_tokens(tokens):
    return [token for token in tokens if token not in russian_stopwords and token != ' ']

df['title'] = df['title'].progress_apply(lambda title: " ".join(filter_tokens(word_tokenize(title, language='russian'))))

In [ ]:
df.head(5)

In [ ]:
df.shape

In [ ]:
# Сначала разделим датафрейм на 3 части по каждой категории.
# Затем создадим три словаря: 1 - список аптеки, 2 - список поставщика, 3 - список для обучения модели (train),
# где каждый словарь будет содержать уникальные пары title_cat: title.

# После этого создадим четвертый словарь, где ключом будет title из первого словаря (список аптеки), 
# а значением - соответствующий title из второго словаря (список поставщика). 
# Этот словарь будет использоваться для оценки корректности сопоставления.

grouped = df.groupby('title_cat')
pharmacy_map, supplier_map, study_dict = {}, {}, {} # 1 - список аптеки, 2 - список поставщика, 3 - список для обучения модели
#четвертый словарь, который связывает title из первого словаря (аптеки) с title из второго (поставщика) по той же категории
title_to_title_true = {}

for name, group in grouped:
    titles = group['title'].tolist()
   
    if len(titles) > 2:
        pharmacy_map[name] = titles[0]
        supplier_map[name] = titles[1]
        
        study_dict[name] = titles[2:]
        title_to_title_true[titles[0]] = titles[1]
    else:
        
        continue

for name, group in grouped:
    titles = group['title'].tolist()
    pharmacy_map[name] = titles[0]

supplier_map[name] = titles[1] if len(titles) > 1 else None

study_dict[name] = titles[2:] if len(titles) > 2 else []

if supplier_map[name] is not None:
    title_to_title_true[titles[0]] = supplier_map[name]

list1_title_cat = list(pharmacy_map.keys())
list1_title = list(pharmacy_map.values())

list2_title_cat = list(supplier_map.keys())
list2_title = list(supplier_map.values())

list3_title_cat = []
list3_title = []

for cat, titles in study_dict.items():
    list3_title_cat.extend([cat] * len(titles))
    list3_title.extend(titles)

pharmacy_list = list(pharmacy_map.items())
supplier_list = list(supplier_map.items())
study_list = [(cat, title) for cat, titles in study_dict.items() for title in titles]
title_to_title_true_list = list(title_to_title_true.items())

pharmacy_df = pd.DataFrame(pharmacy_list, columns=['title_cat', 'title'])
supplier_df = pd.DataFrame(supplier_list, columns=['title_cat', 'title'])
study_df = pd.DataFrame(study_list, columns=['title_cat', 'title'])
title_to_title_true_df = pd.DataFrame(title_to_title_true_list, columns=['Ключ', 'Значение'])

In [ ]:
# посмотрим как выглядит словарь для списка аптеки

pharmacy_df[:5]

In [ ]:
pharmacy_df.shape

In [ ]:
# посмотрим как выглядит словарь для списка поставщика

supplier_df[:5]

In [ ]:
# посмотрим как выглядит словарь для списка для обучения модели

study_df[:5]

In [ ]:
study_df.shape

In [ ]:
# посмотрим как выглядит словарь, который связывает title из первого словаря (аптеки) с title из второго словаря (поставщика) 
# по той же категории

title_to_title_true_df[:5]

In [ ]:
title_to_title_true_df.shape

# Определение train/test

In [ ]:
# в качестве train используем ранее созданный список study (он имеет title и title_cat)
# затем после обучения предсказываем категорию для списка аптеки и списка поставщика

X_train = list3_title # title из списка study
y_train = list3_title_cat # title_cat из списка study
X_test_1 = list1_title # title из списка аптеки
X_test_2 = list2_title # title из списка поставщика
y_test_1 = list1_title_cat # title_cat из списка аптеки 
y_test_2 = list2_title_cat # title_cat из списка поставщика 

In [ ]:
# удаление неиспользуемого df с целью экономии памяти

import gc

# Удаление DataFrame
del df_baza

# Вызов сборщика мусора
gc.collect()

# Наивный байесовский классификатор

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import time
import psutil

process = psutil.Process()

start_time = time.time()
start_memory = process.memory_info().rss

nb = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', MultinomialNB())
])

nb.fit(X_train, y_train)

end_time = time.time()
end_memory = process.memory_info().rss  

y_pred_nb_1 = nb.predict(X_test_1)
y_pred_nb_2 = nb.predict(X_test_2)

print('\nВремя, затраченное на обучение модели: ', end_time - start_time, ' секунд')
print(f"Использовано памяти: {end_memory - start_memory} байт")

In [ ]:
# Расчет Accuracy Score (долю правильно предсказанных наблюдений от общего числа наблюдений)
accuracy_1 = round(accuracy_score(y_test_1, y_pred_nb_1), 2)
accuracy_2 = round(accuracy_score(y_test_2, y_pred_nb_2), 2)

# Расчет Precision (Точность) (доля правильно идентифицированных положительных результатов из всех результатов, которые были классифицированы как положительные)
precision_1 = round(precision_score(y_test_1, y_pred_nb_1, average='weighted', zero_division=0), 2)
precision_2 = round(precision_score(y_test_2, y_pred_nb_2, average='weighted', zero_division=0), 2)

# Расчет Recall (Полнота) (доля правильно идентифицированных положительных результатов из всех реальных положительных результатов)
recall_1 = round(recall_score(y_test_1, y_pred_nb_1, average='weighted', zero_division=0), 2)
recall_2 = round(recall_score(y_test_2, y_pred_nb_2, average='weighted', zero_division=0), 2)

# Расчет F1-score (гармоническое среднее между точностью и полнотой)
f1_1 = round(f1_score(y_test_1, y_pred_nb_1, average='weighted', zero_division=0), 2)
f1_2 = round(f1_score(y_test_2, y_pred_nb_2, average='weighted', zero_division=0), 2)

print('Метрики для списка аптеки')
print('Accuracy score: ', accuracy_1)
print('Precision: ', precision_1)
print('Recall: ', recall_1)
print('F1-score: ', f1_1)

print('\nМетрики для списка поставщика')
print('Accuracy score: ', accuracy_2)
print('Precision: ', precision_2)
print('Recall: ', recall_2)
print('F1-score: ', f1_2)

In [ ]:
# посмотрим визуально первые 10 строк X_test_1, y_pred_nb_1, y_test_1 в виде датафрейма
comparison_df_nb_1 = pd.DataFrame({
    'Имеющийся title из списка аптеки': X_test_1[:10],
    'Прогнозируемая категория для списка аптеки': y_pred_nb_1[:10],
    'Фактическая категория для списка аптеки': y_test_1[:10]
})
comparison_df_nb_1

In [ ]:
# посмотрим визуально первые 10 строк X_test_2, y_pred_nb_2, y_test_2 в виде датафрейма
comparison_df_nb_2 = pd.DataFrame({
    'Имеющийся title из списка поставщика': X_test_2[:10],
    'Прогнозируемая категория для списка поставщика': y_pred_nb_2[:10],
    'Фактическая категория для списка поставщика': y_test_2[:10]
})
comparison_df_nb_2

In [ ]:
# Расчет количества совпадений
matches = sum(1 for i, j in zip(y_pred_nb_1, y_pred_nb_2) if i == j)

# Расчет процента совпадений
matches_percentage = round((matches / len(y_pred_nb_1)) * 100, 2)

print('Процент совпадений: ', matches_percentage)

# Дерево решений

In [ ]:
# удаление неиспользуемой переменной с предыдущего этапа обучения с целью экономии памяти

import gc

del nb
gc.collect() 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import time
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import psutil

process = psutil.Process()

start_time = time.time()
start_memory = process.memory_info().rss 

dt = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', DecisionTreeClassifier())
])

dt.fit(X_train, y_train)

end_time = time.time()
end_memory = process.memory_info().rss  

y_pred_dt_1 = dt.predict(X_test_1)
y_pred_dt_2 = dt.predict(X_test_2)

print('\nВремя, затраченное на обучение модели: ', end_time - start_time, ' секунд')
print(f"Использовано памяти: {end_memory - start_memory} байт")

In [ ]:
# Расчет Accuracy Score (долю правильно предсказанных наблюдений от общего числа наблюдений)
accuracy_1 = round(accuracy_score(y_test_1, y_pred_dt_1), 2)
accuracy_2 = round(accuracy_score(y_test_2, y_pred_dt_2), 2)

# Расчет Precision (Точность) (доля правильно идентифицированных положительных результатов из всех результатов, которые были классифицированы как положительные)
precision_1 = round(precision_score(y_test_1, y_pred_dt_1, average='weighted', zero_division=0), 2)
precision_2 = round(precision_score(y_test_2, y_pred_dt_2, average='weighted', zero_division=0), 2)

# Расчет Recall (Полнота) (доля правильно идентифицированных положительных результатов из всех реальных положительных результатов)
recall_1 = round(recall_score(y_test_1, y_pred_dt_1, average='weighted', zero_division=0), 2)
recall_2 = round(recall_score(y_test_2, y_pred_dt_2, average='weighted', zero_division=0), 2)

# Расчет F1-score (гармоническое среднее между точностью и полнотой)
f1_1 = round(f1_score(y_test_1, y_pred_dt_1, average='weighted', zero_division=0), 2)
f1_2 = round(f1_score(y_test_2, y_pred_dt_2, average='weighted', zero_division=0), 2)

print('Метрики для списка аптеки')
print('Accuracy score: ', accuracy_1)
print('Precision: ', precision_1)
print('Recall: ', recall_1)
print('F1-score: ', f1_1)

print('\nМетрики для списка поставщика')
print('Accuracy score: ', accuracy_2)
print('Precision: ', precision_2)
print('Recall: ', recall_2)
print('F1-score: ', f1_2)

In [ ]:
# посмотрим визуально первые 10 строк X_test_1, y_pred_dt_1, y_test_1 в виде датафрейма
comparison_df_dt_1 = pd.DataFrame({
    'Имеющийся title из списка аптеки': X_test_1[:10],
    'Прогнозируемая категория для списка аптеки': y_pred_dt_1[:10],
    'Фактическая категория для списка аптеки': y_test_1[:10]
})
comparison_df_dt_1

In [ ]:
# посмотрим визуально первые 10 строк X_test_2, y_pred_dt_2, y_test_2 в виде датафрейма
comparison_df_dt_2 = pd.DataFrame({
    'Имеющийся title из списка аптеки': X_test_2[:10],
    'Прогнозируемая категория для списка аптеки': y_pred_dt_2[:10],
    'Фактическая категория для списка аптеки': y_test_2[:10]
})
comparison_df_dt_2

In [ ]:
# Расчет количества совпадений
matches = sum(1 for i, j in zip(y_pred_dt_1, y_pred_dt_2) if i == j)

# Расчет процента совпадений
matches_percentage = round((matches / len(y_pred_dt_1)) * 100, 2)

print('Процент совпадений: ', matches_percentage)

# Вывод списка поставщиков по предсказанным категориям

In [ ]:
# predict_title_cat это предсказанные ранее категории по списку аптеки (в результате обучения DT)
predict_title_cat = y_pred_dt_1 

pharmacy_df_new = pd.DataFrame(predict_title_cat, columns=['predict_title_cat'])
merged_df = pharmacy_df_new.merge(df, left_on='predict_title_cat', right_on='title_cat', how='left')
suppliers_per_cat = merged_df.groupby('predict_title_cat')['manuf_cat'].unique()

suppliers_list = pd.DataFrame({'Категория': suppliers_per_cat.index, 
                               'Поставщики': [', '.join(suppliers) for suppliers in suppliers_per_cat.values]
                              })

In [ ]:
suppliers_list.head(10)

# Случайный лес

In [ ]:
# удаление неиспользуемой переменной с предыдущего этапа обучения с целью экономии памяти

import gc

del dt
gc.collect()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import time
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import psutil

process = psutil.Process()

start_time = time.time()
start_memory = process.memory_info().rss 

rf = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', RandomForestClassifier())
              ])
rf.fit(X_train, y_train)

end_time = time.time()
end_memory = process.memory_info().rss 

y_pred_rf_1 = rf.predict(X_test_1)
y_pred_rf_2 = rf.predict(X_test_2)

print(f"Время выполнения: {end_time - start_time} секунд")
print(f"Использовано памяти: {end_memory - start_memory} байт")

In [ ]:
# Расчет Accuracy Score (долю правильно предсказанных наблюдений от общего числа наблюдений)
accuracy_1 = round(accuracy_score(y_test_1, y_pred_rf_1), 2)
accuracy_2 = round(accuracy_score(y_test_2, y_pred_rf_2), 2)

# Расчет Precision (Точность) (доля правильно идентифицированных положительных результатов из всех результатов, которые были классифицированы как положительные)
precision_1 = round(precision_score(y_test_1, y_pred_rf_1, average='weighted', zero_division=0), 2)
precision_2 = round(precision_score(y_test_2, y_pred_rf_2, average='weighted', zero_division=0), 2)

# Расчет Recall (Полнота) (доля правильно идентифицированных положительных результатов из всех реальных положительных результатов)
recall_1 = round(recall_score(y_test_1, y_pred_rf_1, average='weighted', zero_division=0), 2)
recall_2 = round(recall_score(y_test_2, y_pred_rf_2, average='weighted', zero_division=0), 2)

# Расчет F1-score (гармоническое среднее между точностью и полнотой)
f1_1 = round(f1_score(y_test_1, y_pred_rf_1, average='weighted', zero_division=0), 2)
f1_2 = round(f1_score(y_test_2, y_pred_rf_2, average='weighted', zero_division=0), 2)

print('Метрики для списка аптеки')
print('Accuracy score: ', accuracy_1)
print('Precision: ', precision_1)
print('Recall: ', recall_1)
print('F1-score: ', f1_1)

print('\nМетрики для списка поставщика')
print('Accuracy score: ', accuracy_2)
print('Precision: ', precision_2)
print('Recall: ', recall_2)
print('F1-score: ', f1_2)

In [ ]:
# посмотрим визуально первые 10 строк X_test_1, y_pred_rf_1, y_test_1 в виде датафрейма
comparison_df_rf_1 = pd.DataFrame({
    'Имеющийся title из списка аптеки': X_test_1[:10],
    'Прогнозируемая категория для списка аптеки': y_pred_rf_1[:10],
    'Фактическая категория для списка аптеки': y_test_1[:10]
})
comparison_df_rf_1

In [ ]:
# посмотрим визуально первые 10 строк X_test_2, y_pred_rf_2, y_test_2 в виде датафрейма
comparison_df_rf_2 = pd.DataFrame({
    'Имеющийся title из списка аптеки': X_test_2[:10],
    'Прогнозируемая категория для списка аптеки': y_pred_rf_2[:10],
    'Фактическая категория для списка аптеки': y_test_2[:10]
})
comparison_df_rf_2

In [ ]:
# Расчет количества совпадений
matches = sum(1 for i, j in zip(y_pred_rf_1, y_pred_rf_2) if i == j)

# Расчет процента совпадений
matches_percentage = round((matches / len(y_pred_rf_1)) * 100, 2)

print('Процент совпадений: ', matches_percentage)

# Метод ближайших соседей KNN

In [ ]:
# удаление неиспользуемой переменной с предыдущего этапа обучения с целью экономии памяти

import gc

del rf
gc.collect()

In [ ]:
# подбор параметра n_neighbours

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)

k_values = range(1, 21)
cross_val_scores = []

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(knn, X_train_tfidf, y_train, cv=5)
    cross_val_scores.append(scores.mean())

optimal_k = k_values[cross_val_scores.index(max(cross_val_scores))]
print('Оптимальное значение K: ', optimal_k)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import time
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import psutil

process = psutil.Process()

start_time = time.time()
start_memory = process.memory_info().rss 

knn = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', KNeighborsClassifier(n_neighbors=5))
])

knn.fit(X_train, y_train)

end_time = time.time()
end_memory = process.memory_info().rss 

y_pred_knn_1 = knn.predict(X_test_1)
y_pred_knn_2 = knn.predict(X_test_2)

print('\nВремя, затраченное на обучение модели: ', end_time - start_time, ' секунд')
print(f"Использовано памяти: {end_memory - start_memory} байт")

In [ ]:
# Расчет Accuracy Score (долю правильно предсказанных наблюдений от общего числа наблюдений)
accuracy_1 = round(accuracy_score(y_test_1, y_pred_knn_1), 2)
accuracy_2 = round(accuracy_score(y_test_2, y_pred_knn_2), 2)

# Расчет Precision (Точность) (доля правильно идентифицированных положительных результатов из всех результатов, которые были классифицированы как положительные)
precision_1 = round(precision_score(y_test_1, y_pred_knn_1, average='weighted', zero_division=0), 2)
precision_2 = round(precision_score(y_test_2, y_pred_knn_2, average='weighted', zero_division=0), 2)

# Расчет Recall (Полнота) (доля правильно идентифицированных положительных результатов из всех реальных положительных результатов)
recall_1 = round(recall_score(y_test_1, y_pred_knn_1, average='weighted', zero_division=0), 2)
recall_2 = round(recall_score(y_test_2, y_pred_knn_2, average='weighted', zero_division=0), 2)

# Расчет F1-score (гармоническое среднее между точностью и полнотой)
f1_1 = round(f1_score(y_test_1, y_pred_knn_1, average='weighted', zero_division=0), 2)
f1_2 = round(f1_score(y_test_2, y_pred_knn_2, average='weighted', zero_division=0), 2)

# Выводим результаты
print('Метрики для списка аптеки')
print('Accuracy score: ', accuracy_1)
print('Precision: ', precision_1)
print('Recall: ', recall_1)
print('F1-score: ', f1_1)

print('\nМетрики для списка поставщика')
print('Accuracy score: ', accuracy_2)
print('Precision: ', precision_2)
print('Recall: ', recall_2)
print('F1-score: ', f1_2)

In [ ]:
# посмотрим визуально первые 10 строк X_test_1, y_pred_knn_1, y_test_1 в виде датафрейма
comparison_df_knn_1 = pd.DataFrame({
    'Имеющийся title из списка аптеки': X_test_1[:10],
    'Прогнозируемая категория для списка аптеки': y_pred_knn_1[:10],
    'Фактическая категория для списка аптеки': y_test_1[:10]
})
comparison_df_knn_1

In [ ]:
# посмотрим визуально первые 10 строк X_test_2, y_pred_knn_2, y_test_2 в виде датафрейма
comparison_df_knn_2 = pd.DataFrame({
    'Имеющийся title из списка аптеки': X_test_2[:10],
    'Прогнозируемая категория для списка аптеки': y_pred_knn_2[:10],
    'Фактическая категория для списка аптеки': y_test_2[:10]
})
comparison_df_knn_2

In [ ]:
# Расчет количества совпадений
matches = sum(1 for i, j in zip(y_pred_knn_1, y_pred_knn_2) if i == j)

# Расчет процента совпадений
matches_percentage = round((matches / len(y_pred_knn_1)) * 100, 2)

print('Процент совпадений: ', matches_percentage)

# Метод опорных векторов

In [ ]:
import gc

del knn
gc.collect()

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
import time
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import psutil

process = psutil.Process()

start_time = time.time()
start_memory = process.memory_info().rss

svm_pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', SVC())
])

svm_pipeline.fit(X_train, y_train)

end_time = time.time()
end_memory = process.memory_info().rss 

y_pred_svm_1 = svm_pipeline.predict(X_test_1)
y_pred_svm_2 = svm_pipeline.predict(X_test_2)

print('\nВремя, затраченное на обучение модели: ', end_time - start_time, ' секунд')
print(f"Использовано памяти: {end_memory - start_memory} байт")

In [ ]:
# Расчет Accuracy Score (долю правильно предсказанных наблюдений от общего числа наблюдений)
accuracy_1 = round(accuracy_score(y_test_1, y_pred_svm_1), 2)
accuracy_2 = round(accuracy_score(y_test_2, y_pred_svm_2), 2)

# Расчет Precision (Точность) (доля правильно идентифицированных положительных результатов из всех результатов, которые были классифицированы как положительные)
precision_1 = round(precision_score(y_test_1, y_pred_svm_1, average='weighted', zero_division=0), 2)
precision_2 = round(precision_score(y_test_2, y_pred_svm_2, average='weighted', zero_division=0), 2)

# Расчет Recall (Полнота) (доля правильно идентифицированных положительных результатов из всех реальных положительных результатов)
recall_1 = round(recall_score(y_test_1, y_pred_svm_1, average='weighted', zero_division=0), 2)
recall_2 = round(recall_score(y_test_2, y_pred_svm_2, average='weighted', zero_division=0), 2)

# Расчет F1-score (гармоническое среднее между точностью и полнотой)
f1_1 = round(f1_score(y_test_1, y_pred_svm_1, average='weighted', zero_division=0), 2)
f1_2 = round(f1_score(y_test_2, y_pred_svm_2, average='weighted', zero_division=0), 2)

# Выводим результаты
print('Метрики для списка аптеки')
print('Accuracy score: ', accuracy_1)
print('Precision: ', precision_1)
print('Recall: ', recall_1)
print('F1-score: ', f1_1)

print('\nМетрики для списка поставщика')
print('Accuracy score: ', accuracy_2)
print('Precision: ', precision_2)
print('Recall: ', recall_2)
print('F1-score: ', f1_2)

In [ ]:
# посмотрим визуально первые 10 строк X_test_1, y_pred_svm_1, y_test_1 в виде датафрейма
comparison_df_svm_1 = pd.DataFrame({
    'Имеющийся title из списка аптеки': X_test_1[:10],
    'Прогнозируемая категория для списка аптеки': y_pred_svm_1[:10],
    'Фактическая категория для списка аптеки': y_test_1[:10]
})
comparison_df_svm_1

In [ ]:
# посмотрим визуально первые 10 строк X_test_2, y_pred_svm_2, y_test_2 в виде датафрейма
comparison_df_svm_2 = pd.DataFrame({
    'Имеющийся title из списка аптеки': X_test_2[:10],
    'Прогнозируемая категория для списка аптеки': y_pred_svm_2[:10],
    'Фактическая категория для списка аптеки': y_test_2[:10]
})
comparison_df_svm_2

In [ ]:
# Расчет количества совпадений
matches = sum(1 for i, j in zip(y_pred_svm_1, y_pred_svm_2) if i == j)

# Расчет процента совпадений
matches_percentage = round((matches / len(y_pred_svm_1)) * 100, 2)

print('Процент сопоставлений: ', matches_percentage)

In [ ]:
# удаление неиспользуемой переменной с предыдущего этапа обучения с целью экономии памяти

import gc

del svm_pipeline
gc.collect()

# Решение задачи сопоставления с помощью TF-IDF и косинусного сходства

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
import time
import psutil

process = psutil.Process()

start_time = time.time()
start_memory = process.memory_info().rss

vectorizer = TfidfVectorizer().fit(list1_title + list2_title)
tfidf_matrix1 = vectorizer.transform(list1_title)
tfidf_matrix2 = vectorizer.transform(list2_title)

cosine_similarities = cosine_similarity(tfidf_matrix1, tfidf_matrix2)

print('Косинусные сходства:')
print(cosine_similarities)

thresholds = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

all_precisions = []
all_recalls = []

for threshold in thresholds:
    predicted_pairs = []
    for i in range(len(list1_title)):
        for j in range(len(list2_title)):
            if cosine_similarities[i, j] >= threshold:
                predicted_pairs.append((list1_title[i], list2_title[j]))

    
    predicted_df = pd.DataFrame(predicted_pairs, columns=['Ключ', 'Значение']).drop_duplicates()
    
    true_positive_df = predicted_df.merge(title_to_title_true_df, on=['Ключ', 'Значение'], how='inner')
    false_positive_df = predicted_df.merge(title_to_title_true_df, on=['Ключ', 'Значение'], how='left', indicator=True)
    false_negative_df = title_to_title_true_df.merge(predicted_df, on=['Ключ', 'Значение'], how='left', indicator=True)

    true_positive = len(true_positive_df)
    false_positive = len(false_positive_df[false_positive_df['_merge'] == 'left_only'])
    false_negative = len(false_negative_df[false_negative_df['_merge'] == 'left_only'])

    if true_positive + false_positive > 0:
        precision = true_positive / (true_positive + false_positive)
    else:
        precision = 0.0

    if true_positive + false_negative > 0:
        recall = true_positive / (true_positive + false_negative)
    else:
        recall = 0.0

    all_precisions.append(precision)
    all_recalls.append(recall)
 
    print(f'Порог: {threshold:.1f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0.0:.4f}\n')

all_pairs = [(list1_title[i], list2_title[j]) for i in range(len(list1_title)) for j in range(len(list2_title))]

all_pairs_df = pd.DataFrame(all_pairs, columns=['Ключ', 'Значение'])

end_time = time.time()
end_memory = process.memory_info().rss

y_true = all_pairs_df.merge(title_to_title_true_df, on=['Ключ', 'Значение'], how='left', indicator=True)
y_true['_merge'] = y_true['_merge'].map({'both': 1, 'left_only': 0})
y_true = y_true['_merge'].values
y_scores = cosine_similarities.flatten()    
    
fpr, tpr, thresholds = roc_curve(y_true, y_scores)

roc_auc = auc(fpr, tpr)

print('\nВремя, затраченное на обучение модели: ', end_time - start_time, ' секунд')
print(f"Использовано памяти: {end_memory - start_memory} байт")

In [ ]:
# Добавляем график ROC-кривой
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkblue', lw=2, label=f'ROC curve (area = {roc_auc:.2f})') # Изменен цвет на темно-синий
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show() 
 
# Построение Precision-Recall Curve
plt.figure(figsize=(8, 6))
plt.plot(all_recalls, all_precisions, linestyle='-', color='darkblue', label='Precision-Recall Curve') # Убрали маркеры и изменили цвет
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend()
plt.show() 

# Решение задачи сопоставления с помощью нормализованного расстояния Левенштейна

In [ ]:
import pandas as pd
from rapidfuzz import fuzz
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, precision_recall_curve, auc
import matplotlib.pyplot as plt
import time
import psutil

process = psutil.Process()

start_time = time.time()
start_memory = process.memory_info().rss 

distances = []
for title1 in list1_title:
    for title2 in list2_title:
        distance = 1 - fuzz.ratio(title1, title2) / 100
        distances.append((title1, title2, distance))

distances_df = pd.DataFrame(distances, columns=['list1_title', 'list2_title', 'distance'])

merged_df = distances_df.merge(title_to_title_true_df, left_on=['list1_title', 'list2_title'], right_on=['Ключ', 'Значение'], how='left')
merged_df['true_match'] = merged_df['Ключ'].notna().astype(int)

thresholds = [0.2, 0.25, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]
metrics = []

for threshold in thresholds:
    merged_df['pred_match'] = (merged_df['distance'] <= threshold).astype(int)
    precision = precision_score(merged_df['true_match'], merged_df['pred_match'])
    recall = recall_score(merged_df['true_match'], merged_df['pred_match'])
    f1 = f1_score(merged_df['true_match'], merged_df['pred_match'])
    metrics.append((threshold, precision, recall, f1))

metrics_df = pd.DataFrame(metrics, columns=['Threshold', 'Precision', 'Recall', 'F1 Score'])

end_time = time.time()
end_memory = process.memory_info().rss 

print('\nВремя, затраченное на обучение модели: ', end_time - start_time, ' секунд')
print(f"Использовано памяти: {end_memory - start_memory} байт")

fpr, tpr, roc_thresholds = roc_curve(merged_df['true_match'], -merged_df['distance'])
roc_auc = auc(fpr, tpr)

precision, recall, pr_thresholds = precision_recall_curve(merged_df['true_match'], -merged_df['distance'])
pr_auc = auc(recall, precision)

plt.figure(figsize=(16, 6))

# ROC Curve
plt.subplot(1, 2, 1)
plt.plot(fpr, tpr, color='darkblue', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend(loc='lower right')
plt.show() 

plt.figure(figsize=(16, 6))

# Precision-Recall Curve
plt.subplot(1, 2, 2)
plt.plot(recall, precision, color='darkblue', lw=2, label=f'PR curve (area = {pr_auc:.2f})') 
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve')
plt.legend(loc='lower left')

plt.tight_layout()
plt.show() 

print(metrics_df)